In [ ]:
!pip install xlsxwriter

import pandas as pd
import numpy as np
from google.colab import files
import os

In [ ]:
data = {
'Name': ['Employee_1', 'Employee_2', 'Employee_3', 'Employee_4', 'Employee_5',
'Employee_6', 'Employee_7', 'Employee_8', 'Employee_9', 'Employee_10',
'Employee_11', 'Employee_12', 'Employee_13', 'Employee_14', 'Employee_15',
'Employee_16', 'Employee_17', 'Employee_18', 'Employee_19', 'Employee_20',
'Employee_21', 'Employee_22', 'Employee_23', 'Employee_24', 'Employee_25',
'Employee_26', 'Employee_27', 'Employee_28', 'Employee_29', 'Employee_30'],
'Salary': [54895.0, 38784.0, 120000.0, 60309.0, 46788.0,
50598.0, 61915.0, 50651.0, 38155.0, 55480.0,
49396.0, 64183.0, 38894.0, 46672.0, 20000.0,
57935.0, 57055.0, 44719.0, 66267.0, 48339.0,
44490.0, 48817.0, 39738.0, 59433.0, 80823.0,
80000.0, 52704.0, 54168.0, 56569.0, 45481.0],
'Age': [49, 48, 34, 33, 59, 36, 34, 30, 36, 29,
29, 54, 55, 29, 53, 34, 23, 41, 58, 41,
45, 22, 39, 47, 22, 30, 53, 33, 23, 52],
'Function': ['HR', 'Sales', 'Manager', 'Manager', 'HR',
'Sales', 'Manager', 'HR', 'Sales', 'HR',
'Sales', 'Developer', 'Developer', 'Sales', 'HR',
'Sales', 'Developer', 'Developer', 'HR', 'Sales',
'HR', 'Manager', 'HR', 'Sales', 'HR', 'Sales',
'Sales', 'Developer', 'Developer', 'HR']
}

df = pd.DataFrame(data)

In [ ]:
# Age Bucket
age_bins = [0, 20, 30, 40, 50, 100]
age_labels = ['<20 years', '20-30 years', '30-40 years', '40-50 years', '>50 years']
df['Age_Bucketed'] = pd.cut(df['Age'], bins=age_bins, labels=age_labels, right=False)
department_mapping = {
'HR': 'HR001',
'Sales': 'SAL002',
'Manager': 'MGR003',
'Developer': 'DEV004'
}
df['Function_Code'] = df['Function'].map(department_mapping)

# Outliers
Q1 = df['Salary'].quantile(0.25)
Q3 = df['Salary'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df['Outlier_Status'] = np.where(
    (df['Salary'] < lower_bound) | (df['Salary'] > upper_bound),
    'Outlier',
    'Not an Outlier'
)

In [ ]:
# dictionary to dataFrame
department_df = pd.DataFrame(list(department_mapping.items()),
                             columns=['Department', 'Department_Code'])

filename = 'employee_data.xlsx'

with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:

    df.to_excel(writer, sheet_name='Employee_Data', index=False)
    department_df.to_excel(writer, sheet_name='Department_Codes', index=False)

    workbook = writer.book

    worksheet = writer.sheets['Employee_Data']
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(0, col_num, value, header_format)

    worksheet.set_column('A:A', 12)  # Name
    worksheet.set_column('B:B', 10)  # Salary
    worksheet.set_column('C:C', 6)   # Age
    worksheet.set_column('D:D', 12)  # Function
    worksheet.set_column('E:E', 12)  # Age_Bucketed
    worksheet.set_column('F:F', 12)  # Function_Code
    worksheet.set_column('G:G', 15)  # Outlier_Status

    outlier_rows = df[df['Outlier_Status'] == 'Outlier'].index.tolist()
    for row_idx in outlier_rows:
        worksheet.set_row(row_idx + 1, None, outlier_format)

    worksheet = writer.sheets['Department_Codes']
    for col_num, value in enumerate(department_df.columns.values):
        worksheet.write(0, col_num, value, header_format)

    worksheet.set_column('A:B', 15)

if os.path.exists(filename):
    files.download(filename)
    print(f"Excel file has been created and downloaded as '{filename}'")
else:
    print(f"Error: File '{filename}' was not created properly")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Excel file has been created and downloaded as 'employee_data.xlsx'
